In [1]:
from datasets import load_from_disk
import pandas as pd
import re
import numpy as np

2021-11-01 16:12:23.791374: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
DATASET_PATH = '/opt/ml/data/train_dataset'
WIKI_JSON_PATH = '/opt/ml/data/wikipedia_documents.json'

In [3]:

dataset = load_from_disk(DATASET_PATH)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 6059
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 319
    })
})

In [30]:
dataset_train = dataset["train"].to_pandas()

## Train 살펴보기

In [6]:
ids = []
answers = []
contexts = []
questions = []
titles = []
document_ids = []

for ex in dataset['train']:
    ids.append(ex['id'])
    answers.append(ex['answers'])
    contexts.append(ex['context'])
    questions.append(ex['question'])
    titles.append(ex['title'])
    document_ids.append(ex['document_id'])


train_df = pd.DataFrame({"id": ids,"question" : questions, "answer": answers, "context" : contexts, "title" : titles, "doc_id" : document_ids})


In [7]:
train_df

,id,question,answer,context,title,doc_id
0,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,"{'answer_start': [235], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,미국 상원,18293
1,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,인사조직관리,51638
2,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,"{'answer_start': [510], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제,5028
3,mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,"{'answer_start': [625], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",금동삼존불감,34146
4,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구,47334
...,...,...,...,...,...,...
3947,mrc-0-005285,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,"{'answer_start': [317], 'text': ['지질 구조']}",이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,이오 (위성),8275
3948,mrc-1-000699,애니는 어디서 태어났는가?,"{'answer_start': [616], 'text': ['뉴저지']}",애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,내니 다이어리,37764
3949,mrc-0-003429,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,"{'answer_start': [255], 'text': ['칼 에케르트']}",1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,빈 필하모니 관현악단,12730
3950,mrc-0-003956,드래곤의 암수 구분이 있는 신화는?,"{'answer_start': [476], 'text': ['슬라브 신화']}",원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤,10924


In [8]:
train_df.describe()

,doc_id
count,3952.000000
mean,28399.783401
std,17475.051830
min,4572.000000
25%,11615.750000
50%,26295.000000
75%,44240.250000
max,60603.000000


### answer 특수문자 확인

In [9]:
ans_text = []
for ans in train_df["answer"]:
    ans_text.append(*ans['text'])
ans_text

['하원',
 '《경영의 실제》',
 '백성',
 '중국',
 '4개',
 '드래곤',
 '형양태수 왕식',
 '이탈리아군',
 '큰아들 유',
 '왕대마을',
 '음독자살',
 '출장 잦은 건축가',
 '반신화적인 인물인 우파',
 '1951년',
 '예수',
 "'초일기'",
 '1916년',
 '레드삭스',
 '삼판동',
 '다산 정약용',
 '대나라',
 '10달러',
 '1945년',
 '사립 김노인 고등학교',
 '체벽에 잇는 감지기',
 '근대철학',
 '215km',
 '마커트리로 장식을 한 트롱프 뢰유',
 '아르투로',
 '조립 라인',
 '우전왕(優塡王)식 착의법',
 '친구',
 '2015년',
 '케이스 커크',
 '직사각형',
 '상인',
 '중국',
 '지상요원들',
 '시조 거인',
 "'발라즈 찬 카이이'",
 '1964년 12월 12일',
 'NADPH',
 '그리고리 신부(Father Grigori)',
 '헌법 제12조 제3항',
 '1907년까지',
 '부근 북동부',
 '덱',
 '평사낙안형',
 '집정관',
 '2004년',
 '2002',
 '빌 비크',
 '사대부',
 '19 K 이하',
 '이탈리아 피아트 사',
 '임수정',
 '깨진 술잔',
 '공산주의',
 '≪인간의 이해: 개념의 집단적 사용 및 진화 (1972)≫',
 '경례',
 '상원',
 '14세기',
 '〈마브루공(公)〉',
 '마수드의 군대',
 '라틴어',
 '핑둥현',
 '나고야',
 '1753년',
 '새로 조직된 훈련대의 제1대 대대장',
 '장시성',
 '왕권',
 '표트르 1세',
 '하세가와 마나미',
 '독립 선언식',
 '평형상태',
 '리더',
 '〈성당과 시장〉(The Cathedral and the Bazaar)',
 '바순',
 '골',
 '마케도니아의 장군 알렉산더 3세',
 '2000년',
 "'길버트 에드워드'",
 '소련',
 '오아하카주',
 '〈목포의 눈물〉',
 '일본',
 '상사병',
 '송기중 교수',
 '의

In [10]:
processed_text = []
for idx, text in enumerate(ans_text):
    tmp = re.sub(r"[가-힣]+|[0-9]+|[ ]+|[a-z]+|[A-Z]+|[ㄱ-ㅎ]+","",text)
    if tmp != '':
        processed_text.append((idx,tmp))
processed_text

[(1, '《》'),
 (15, "''"),
 (30, '(優塡王)'),
 (39, "''"),
 (42, '()'),
 (58, '≪:()≫'),
 (62, '〈(公)〉'),
 (76, '〈〉()'),
 (81, "''"),
 (84, '〈〉'),
 (123, '()'),
 (136, '‘’(前意識)'),
 (137, '()'),
 (146, '%'),
 (150, '《》'),
 (165, '()'),
 (183, '%'),
 (188, '《》'),
 (193, '()'),
 (195, "''"),
 (219, "''"),
 (228, '(南雲忠一)'),
 (269, '(魯陵)'),
 (272, '《》'),
 (287, '()'),
 (295, '.'),
 (304, '()'),
 (343, '()'),
 (363, '(塔身)'),
 (366, ','),
 (372, '()'),
 (381, '‘’'),
 (389, '.'),
 (392, '(大)'),
 (396, '()'),
 (431, '.'),
 (433, '()'),
 (476, '《》'),
 (478, '‘’'),
 (500, '()'),
 (520, '"...!!"'),
 (525, '(片刀)'),
 (544, '\xad\xad'),
 (557, '-'),
 (562, "''"),
 (579, '""'),
 (608, '""'),
 (628, "''"),
 (636, '.'),
 (640, '.'),
 (648, '()'),
 (650, '()'),
 (706, "''"),
 (713, '.'),
 (733, '(陳理)'),
 (741, ','),
 (744, '‘’'),
 (745, '“”'),
 (747, '()'),
 (789, '(-)'),
 (790, '()'),
 (812, '··'),
 (820, '%'),
 (839, '(-)'),
 (842, '〈〉'),
 (888, ','),
 (896, '〈〉'),
 (920, '<>'),
 (935, '‘’'),
 (962, '-'),
 (9

In [11]:
# '\xad\xad' in processed_text[:,1]
processed_text = np.array(processed_text)
cnt=0
for tmp in sorted(list(set(''.join(processed_text[:,1])))):
    cnt += 1
    print(tmp, end=' ')
len(set(''.join(processed_text[:,1])))

! " % ' ( ) + , - . / : < > ~ ­ ° · ä ‘ ’ “ ” ∧ ≪ ≫ 〈 〉 《 》 「 」 『 』 い う し な に の よ ら ・ 一 之 二 京 信 優 儼 公 冬 刀 分 前 割 劉 務 勳 北 匡 南 卵 原 受 呂 坂 城 堂 塔 塡 壇 夔 大 太 奎 媒 孟 學 宋 害 寺 平 康 式 張 弼 循 心 忠 念 性 意 愛 成 扶 文 日 月 有 格 槪 次 毅 池 汪 河 流 海 淵 淸 片 牲 狼 王 理 琿 甚 田 監 祚 祠 祭 縣 總 肥 臺 興 華 蓮 蔣 藏 識 資 超 趙 身 軍 輪 辣 遷 部 郵 鄭 釋 重 金 鉱 限 陳 陵 陽 雲 餘 駅 魯 麻 ＜ ＞ 

159

In [12]:
len("! \" % \' ( ) + , - . / : < > ~ ­ ° · ä  ‘ ’ “ ” ∧ ≪ ≫ 〈 〉 《 》 「 」 『 』 い う し な に の よ ら ・ 一 之 二 京 信 優 儼 公 冬 刀 分 前 割 劉 務 勳 北 匡 南 卵 原 受 呂 坂 城 堂 塔 塡 壇 夔 大 太 奎 媒 孟 學 宋 害 寺 平 康 式 張 弼 循 心 忠 念 性 意 愛 成 扶 文 日 月 有 格 槪 次 毅 池 汪 河 流 海 淵 淸 片 牲 狼 王 理 琿 甚 田 監 祚 祠 祭 縣 總 肥 臺 興 華 蓮 蔣 藏 識 資 超 趙 身 軍 輪 辣 遷 部 郵 鄭 釋 重 金 鉱 限 陳 陵 陽 雲 餘 駅 魯 麻 ＜ ＞".split(' '))

160

In [13]:
# '\xad\xad' in processed_text[:,1]
train_answer_special_characters = set(''.join(processed_text[:,1]))
train_answer_special_characters

{'!',
 '"',
 '%',
 "'",
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 '<',
 '>',
 '~',
 '\xad',
 '°',
 '·',
 'ä',
 '‘',
 '’',
 '“',
 '”',
 '∧',
 '≪',
 '≫',
 '〈',
 '〉',
 '《',
 '》',
 '「',
 '」',
 '『',
 '』',
 'い',
 'う',
 'し',
 'な',
 'に',
 'の',
 'よ',
 'ら',
 '・',
 '一',
 '之',
 '二',
 '京',
 '信',
 '優',
 '儼',
 '公',
 '冬',
 '刀',
 '分',
 '前',
 '割',
 '劉',
 '務',
 '勳',
 '北',
 '匡',
 '南',
 '卵',
 '原',
 '受',
 '呂',
 '坂',
 '城',
 '堂',
 '塔',
 '塡',
 '壇',
 '夔',
 '大',
 '太',
 '奎',
 '媒',
 '孟',
 '學',
 '宋',
 '害',
 '寺',
 '平',
 '康',
 '式',
 '張',
 '弼',
 '循',
 '心',
 '忠',
 '念',
 '性',
 '意',
 '愛',
 '成',
 '扶',
 '文',
 '日',
 '月',
 '有',
 '格',
 '槪',
 '次',
 '毅',
 '池',
 '汪',
 '河',
 '流',
 '海',
 '淵',
 '淸',
 '片',
 '牲',
 '狼',
 '王',
 '理',
 '琿',
 '甚',
 '田',
 '監',
 '祚',
 '祠',
 '祭',
 '縣',
 '總',
 '肥',
 '臺',
 '興',
 '華',
 '蓮',
 '蔣',
 '藏',
 '識',
 '資',
 '超',
 '趙',
 '身',
 '軍',
 '輪',
 '辣',
 '遷',
 '部',
 '郵',
 '鄭',
 '釋',
 '重',
 '金',
 '鉱',
 '限',
 '陳',
 '陵',
 '陽',
 '雲',
 '餘',
 '駅',
 '魯',
 '麻',
 '＜',
 '＞'}

In [14]:
processed_text = np.array(processed_text)
print(len(set(processed_text[:,1])))
for text in set(processed_text[:,1]):
    print(text,end=", ")
# print(set(processed_text[:,1]))


94
“”, '(海蓮之塔)', 〈〉, (受信), (呂超), ‘’(鉱害), “..”, (ä), (-), (日坂駅), ‘,’, (淵太祚), +, (張勳), (南雲忠一), ­­, (興輪寺), ', (華淸池), 《》(), ",", "", (重), (蔣儼), ‘(一心)’, ·, “()”, (汪), ‘∧’, (劉肥), !, (有限性), ,, ≪:()≫, (祭壇), 〈(公)〉, “(成田のようにならないようにしよう)”, ≪≫, (金學奎), (原田甚次), (麻辣), ‘’(前意識), %, (流), <>, (宋公祠), ., 『』, (遷), ‘’, 《()》, (循資格式), ＜＞, (軍毅), (塔身), (冬牲), °, 《》, ・, (大), 「」, (釋總), (片刀), (愛琿城,), (優塡王), '', (魯陵), -, (), (文藏臺), (趙匡), 〈〉(), ··, -%, .%, (陳理), .., (扶餘縣監), (北京), (,二月河), (郵務學堂), (華陽), "...!!", ",(部分割卵)", "()", .(.), (媒槪念), /, (鄭夔弼), :, ~, (狼孟), ""(), (康平)(), 

In [15]:
processed_text = np.array(processed_text)
print(len(set(processed_text[:,1])))
for text in set(processed_text[:,1]):
    print(text,end=", ")
# print(set(processed_text[:,1]))


94
“”, '(海蓮之塔)', 〈〉, (受信), (呂超), ‘’(鉱害), “..”, (ä), (-), (日坂駅), ‘,’, (淵太祚), +, (張勳), (南雲忠一), ­­, (興輪寺), ', (華淸池), 《》(), ",", "", (重), (蔣儼), ‘(一心)’, ·, “()”, (汪), ‘∧’, (劉肥), !, (有限性), ,, ≪:()≫, (祭壇), 〈(公)〉, “(成田のようにならないようにしよう)”, ≪≫, (金學奎), (原田甚次), (麻辣), ‘’(前意識), %, (流), <>, (宋公祠), ., 『』, (遷), ‘’, 《()》, (循資格式), ＜＞, (軍毅), (塔身), (冬牲), °, 《》, ・, (大), 「」, (釋總), (片刀), (愛琿城,), (優塡王), '', (魯陵), -, (), (文藏臺), (趙匡), 〈〉(), ··, -%, .%, (陳理), .., (扶餘縣監), (北京), (,二月河), (郵務學堂), (華陽), "...!!", ",(部分割卵)", "()", .(.), (媒槪念), /, (鄭夔弼), :, ~, (狼孟), ""(), (康平)(), 

In [16]:
train_df.iloc[1030]["context"]

"청자 상감국화모란유로죽문 매병은 과형(瓜形)처럼 몸체를 6개의 면으로 음각하여 분할하였다. 상하 종속문으로 뇌문과 연판문을 상감하고 주제문으로 몸체 가운데 흑백 상감으로 된 능화형 창 안에 다양한 문양 조합을 표현하였다. 문양 조합은 세 종류로 먼저 버드나무와 수금, 새가 나무 위에 앉아 있는 형식, 대나무나 갈대와 수금의 형식과 마지막으로 닥꽃이나 모란, 혹은 국화에 수금과 나방을 결합하는 형식이다. 이런 과형 매병의 성형 방식과 다양한 문양 조합은 13세기 이후의 새로운 변화로 고려청자의 장식성이 더욱 심화되는 증표로 삼을 수 있다. 유색은 유약이 골고루 녹지 않아 선명하지 않은 부분이 있고 일부 유약의 핀 홀 현상과 뭉침 현상이 보인다.\\n\\n제작시기는 과형의 형태와 문양 조합, 상감 기법의 다양한 사용, 유색의 변화 등으로 미루어 당연히 음각매병과는 편차가 있을 것으로 추정해 볼 수 있지만 동시에 출토된 유물이므로 확정할 수는 없다. 역으로 13세기를 전후해서 고려청자의 다양한 기형과 장식이 공존했음을 보여주는 좋은 예이다.\\n\\n죽찰에는 '준'(樽)이라는 기명의 명칭과 내용물이 기재되어 있어 고려시대 매병의 표기와 참기름을 담았던 매병의 당시 용도를 알 수 있게 하는 시대성과 희소성을 지니고 있다. 상감매병과 함께 출토된 죽찰은 ‘ 〉〈’ 모양의 홈을 경계로 머리와 몸통으로 구분할 수 있다. 홈은 죽찰을 매병에 매달기 위해 판 것이다. 머리 상단을 ‘∧’ 모양으로 다듬어 마치 꽃봉오리 같다. 앞면과 뒷면 모두 글자가 있으며, 앞면은 8자, 뒷면은 6자 총 14자다. 앞면에는 “중방도장교오문부(重房都將校吳文富)”가 적혀 있는데 수취인을 적은 것이다. 뒷면에는 “택상진성준봉(宅上眞盛樽封)”이라고 쓰여 있어 준(樽)에 참기름을 담아 올린다는 내용이다."

In [17]:
uncommon_datas = pd.DataFrame(train_df,index=[i for i, _ in processed_text])
uncommon_datas.to_csv('./uncommon_answers.csv')

In [18]:
tmp = uncommon_datas['context']
tmp

1       NaN
15      NaN
30      NaN
39      NaN
42      NaN
       ... 
3832    NaN
3857    NaN
3873    NaN
3928    NaN
3946    NaN
Name: context, Length: 337, dtype: object

In [19]:
print(train_df.iloc[544]["context"])
print(train_df.iloc[544]["answer"])

경상북도 출신으로 1935년에 경성제국대학 법문학부 법학과를 졸업했다. 1938년에 경성지방법원 및 경성지방법원검사국 사법관시보가 된 것을 시작으로 법조계에서 근무를 시작했다.\n\n1939년에는 경성지방법원 검사대리가 되었고, 1940년에 예비검사를 거쳐 그해부터 1943년까지 조선총독부 검사로 근무했다. 강중인은 경제담당 검사로 일하면서 태평양 전쟁 중 시국 연설과 기고로 사회 활동을 병행했다. \n\n1942년에 조선임전보국단 주최로 경성중앙방송국에서 저명 인사의 시국연설을 방송했을 때, 〈필승사상전(必勝思想戰)〉이라는 제목으로 연설한 일이 있다.\n\n《삼천리》에 〈최근의 반도의 경제사범 ­국민의 신경제 윤리의 파악을 위하여­〉라는 글을 기고하기도 했다. 이 글에서는 경제사범을 "미증유의 국난"을 돌파해 "대동아를 건설하는" 국가적으로 중대한 순간에 사리사욕에만 눈이 먼 집단으로 묘사하고, 이들은 국민의 일대수치이므로 엄벌주의로 대처할 것이라 경고했다. 또한 관민이 일체가 되어 "일사보국의 성을 다함"으로써 성업을 어서 완수해야 한다는 주장을 폈다.\n\n1943년에 대전지방법원 검사로 자리를 옮겨 일본 제국이 태평양 전쟁에서 패망할 때까지 근무했다. \n\n미군정 하의 서울에서 변호사를 개업하면서 좌익 운동에 뛰어들었다. 좌익 법조인 모임인 법학자동맹과 남조선로동당에 가입해 활동했다. 강중인이 남로당에 가입하게 된 동기는 후에 법정에서 다음과 같이 밝힌 바 있다. \n\n일정(日政) 당시에는 내가 내 한 몸을 구하기에 여력이 없었습니다. 8·15 해방을 맞이하자 피고는 이때야 깨달은 바가 있었습니다. 이는 다름이 아닙니다. 내가 어찌하면 국가와 민족을 위하여 일할 수 있을까 함이었습니다. 해외에서 국가와 민족을 위하여 투쟁하던 위대한 애국자들이 해방된 조국을 찾아 들어옴을 볼 때 무어라고 표현할 수 없는 느낌이 있었습니다. 그 후 피고는 남로당에 가입을 작정하였던 것이며, 법맹에도 초창기부터 가입하였습니다.\n\n1946년 민주주의민족전선 토지문제연구

In [20]:
print(train_df.iloc[544])

id                                               mrc-0-001312
question                    강중인이 경제사범을 비판하며 국민의 일대수치라 표현한 글은?
answer      {'answer_start': [324], 'text': ['최근의 반도의 경제사범...
context     경상북도 출신으로 1935년에 경성제국대학 법문학부 법학과를 졸업했다. 1938년에...
title                                                     강중인
doc_id                                                  15575
Name: 544, dtype: object


In [21]:
len(train_df["doc_id"]), len(train_df["doc_id"].unique())

(3952, 3340)

In [22]:
train_df[train_df["doc_id"].duplicated()]

,id,question,answer,context,title,doc_id
68,mrc-0-001698,을미사변 직후에 이범래는 무엇에 임명되었나요?,"{'answer_start': [144], 'text': ['새로 조직된 훈련대의 ...",1894년 장위영 영관을 지낸 무관이었다. 1896년 명성황후가 일본인 낭인들에게 ...,이범래 (1868년),14083
292,mrc-0-005237,마리아는 어느 나라에 계속 머물러 있었나요?,"{'answer_start': [706], 'text': ['이탈리아']}",하느님이시요 사람이신 그리스도의 시\n1943년부터 1951년까지 마리아 발토르타는...,마리아 발토르타,31222
320,mrc-0-005401,연병호의 형제가 사망한 날짜는?,"{'answer_start': [1327], 'text': ['1926년 5월 14...",1878년 10월 21일 조선 충청북도 증평에서 태어난 그는 형제로는 독립운동가 연...,연병환,53017
327,mrc-0-002297,"논리형, 제한형 프로그래밍 언어와 대조되는 것은?","{'answer_start': [510], 'text': ['명령형 언어']}","선언형 프로그래밍은 두 가지 뜻으로 통용되고 있다.\n\n한 정의에 따르면, 프로그...",선언형 프로그래밍,8428
337,mrc-0-005356,조선미술가동맹 조선화분과에서 김용준이 맡은 직책은?,"{'answer_start': [680], 'text': ['위원장']}",경북 선산에서 태어나 경성부의 중앙고등보통학교를 졸업했다. 화가 이종우에게 수학한 ...,김용준 (1904년),9868
...,...,...,...,...,...,...
3943,mrc-0-004029,2012년에 부일장학회의 주식반환에 대해 기각 결정을 내린 재판부는?,"{'answer_start': [329], 'text': ['서울중앙지법 민사합의1...",진실규명 결정을 받은 김지태의 유가족들은 2010년 6월에야 법원에 정수장학회와 국...,부일장학회,22495
3946,mrc-0-004590,영류왕이 천리장성의 건축 총괄 책임을 맡겼던 인물은 누구인가?,"{'answer_start': [474], 'text': ['연태조(淵太祚)']}","628년에 당이 마지막 남은 군벌 세력과 서돌궐을 제거하고 통일하였듯, 당의 성장을...",막리지의 난,59702
3947,mrc-0-005285,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,"{'answer_start': [317], 'text': ['지질 구조']}",이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,이오 (위성),8275
3950,mrc-0-003956,드래곤의 암수 구분이 있는 신화는?,"{'answer_start': [476], 'text': ['슬라브 신화']}",원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤,10924


In [23]:
train_df[train_df["title"] == "연병환"].iloc[0]["context"][1327:1327+12]

'1926년 5월 14일'

### train context에 존재하는 특수문자 살펴보기 

In [24]:
train_contexts = train_df["context"]

In [25]:
processed_text = []
for idx, text in enumerate(train_contexts):
    tmp = re.sub(r"\\n+|\'|[가-힣]+|[0-9]+|[ ]+|[a-z]+|[A-Z]+|[ㄱ-ㅎ]+","",text)
    if tmp != '':
        processed_text.append((idx,tmp))
processed_text

[(0, '()...,/..,.,.,,.()..()....=--'),
 (1, '.,.,,,,.《》,.,..(),.,.,,.'),
 (2, ',.....《》.()〈〉(告別上諭),“,.”.“,,,,,.”..'),
 (3, ',(佛龕).,.,.,..,.(光背),.(冠)..,,(冠)∼..'),
 (4, ',.(外函),.“癸巳二月日施主承表兩主”.,.,,,.'),
 (5, '..,.《》...,.,,,,,.,.,,.,.'),
 (6, '.,,,.(館驛),‘(更)’.‘.’.,(天人)......,..(),(漢中王)(樊城)...'),
 (7, '...,...=...,,,.,....'),
 (8, '.,,..,(田倉初子).,,.,.(田倉仁).,....'),
 (9,
  '(母后山)...,,,.(蘿蔔山).(母護山)......,,.,.,."".,,."".?,""..""."","".,""(),"".(),.,,..'),
 (10, ',...,.,()()..,,.,.,..,,.,.'),
 (11, '...,,...,.,.,....,..,.....,...,..,.....,.,..'),
 (12, ',,.,..,.,,().,....,.'),
 (13, '..,“”,“(李淳鎔),”,...,....'),
 (14, '.,.:()..(),..,.,.,..,.'),
 (15, '.,..,(,,,,),(,,,),,,,,.,(草日記),'),
 (16, '.~,,....,..-.,,,..,..()..,..()...,.,,,.'),
 (17, '.",.."..,....,.....'),
 (18, '..,.,.[前村][梅田].,.,,,(異說).'),
 (19, ',《》,《》,《》.,..,...(正祖),.,.,,...'),
 (20, ',.,.,(恒山).(,,)..,,(),,..,,.(),,'),
 (21, '.,.,...“!..,.,..,,.,..'),
 (22, '.,,..""....,,,..'),
 (23, '.,...,,,,....,.,.,..,.....,..,.,.,,,.'),
 (24, '...,

In [26]:
processed_text = np.array(processed_text)
cnt=0
for tmp in sorted(list(set(''.join(processed_text[:,1])))):
    cnt += 1
    print(tmp, end=' ')
len(set(''.join(processed_text[:,1])))

! " # $ % & ( ) * + , - . / : ; < = > ? [ ] ^ _ ` { | } ~ ­ ° ± ² ´ µ · ¹ ¾ É Ó × Þ ß à á ä æ ç è é ê ë ì í ð ó ö ù ú ü ý ā ă ć Đ đ ē ğ ī ł œ ū ŭ ơ ư ǫ ɔ ə ɛ ʲ ˈ ː ́ ΄ Α Δ Ε Κ Λ Μ Π Σ Φ ά έ ή ί α β γ δ ε η θ ι κ λ μ ν ξ ο π ρ ς τ υ φ χ ω ό ύ ώ І В Л М Н П Р С Ю а в д е з и й к л м н о п р с т х ч ш ы я і ִ ַ ׁ ׇ ח י מ ש إ ا ب ة ت ح ر ز س ش ص ع غ ق ك ل م ه و ي ی क त र ल स ़ ा ि ी ु ত র স া ে ச ஜ ம ய ர ழ ா ோ ் ა გ დ ე ვ თ ი ნ ო ს ც ხ ᐨ ᚢ ᚴ ᛁ ᛚ ạ ầ ờ ữ Ἰ ὁ ὔ ‎ – — ― ‘ ’ “ ” • ․ … ‧ ℃ Ⅰ Ⅱ ← ↑ → ↓ ⇌ ∧ ∼ ≪ ≫ ① ② ③ ④ ⑤ ⑥ ⑦ ⑧ ⑨ ⑪ □ ▲ △ ○ ☆ ☞ ♣ 　 、 。 〈 〉 《 》 「 」 『 』 【 】 〔 〕 あ い う か が き こ ご さ し だ ち つ づ て な に の は び み め よ ら り る を ん ァ ア イ ウ ェ オ カ キ ク ゲ コ サ シ ジ ス タ ダ チ ッ ツ ト ド ニ ネ ノ ハ バ パ ヒ ビ ピ フ マ ム メ モ ャ ヨ ラ ル レ ロ ヲ ン ヶ ・ ー ㅏ ㅕ ㅚ ㅟ ㆍ ㈜ ㎜ ㎝ ㎞ ㎠ ㎡ 一 丁 七 万 丈 三 上 下 不 丑 丕 世 丘 丙 丞 両 中 丸 丹 主 丽 乃 之 乘 乙 九 乱 乾 事 二 五 井 亡 交 亥 享 京 亭 人 仁 仇 今 他 仗 付 仙 代 令 仲 件 任 企 伊 伍 休 会 伯 伴 伸 伺 佃 位 低 住 体 何 佛 作 佩 使 來 例 侍 侗 依 侯 俊 俗 俘 保 信 俣 修 倉 倍 倒 候 借 倩 倫 偃 假 偉 停 健 側 傅 傍 備 傳 像 僑 僚 僧 價 儀 儒 償 優 儼 兀 允 元 兆 先 光 克 兌 免 兒 党 兢 入 內 全 兩 兪 八 

2083

In [27]:
train_context_special_characters = set(''.join(processed_text[:,1]))
train_context_special_characters

{'宋',
 '休',
 '烹',
 '点',
 '考',
 '芝',
 'თ',
 '民',
 '璽',
 '妻',
 '杯',
 '猪',
 '²',
 '覇',
 'Φ',
 '超',
 '秉',
 '吾',
 'ī',
 '衡',
 '潔',
 '彪',
 '人',
 'ん',
 '就',
 '益',
 '匠',
 '清',
 '兆',
 '止',
 '〔',
 '前',
 '坂',
 '旣',
 '彰',
 '胤',
 '混',
 'Μ',
 'ی',
 '脈',
 'غ',
 '元',
 '東',
 'х',
 'ச',
 '翅',
 '籍',
 '姬',
 '凹',
 '拉',
 '形',
 '母',
 '原',
 '私',
 '娘',
 '峴',
 '隱',
 '手',
 '自',
 '營',
 '忠',
 'ầ',
 '面',
 '顧',
 ']',
 '鎌',
 '鼻',
 '郳',
 'い',
 '傍',
 '狀',
 '圈',
 '・',
 '^',
 '周',
 '雙',
 'ル',
 '倩',
 '他',
 '孟',
 '磨',
 '內',
 '兪',
 '落',
 '粒',
 'ư',
 '弼',
 '蔣',
 '勤',
 '擧',
 '寫',
 'ヶ',
 'र',
 '炯',
 '麵',
 '藩',
 '碗',
 '詞',
 '烈',
 '症',
 '袁',
 '役',
 '坊',
 '焼',
 '筆',
 '按',
 '康',
 '権',
 '夜',
 'Κ',
 '栗',
 '丹',
 '苦',
 '路',
 '史',
 '赤',
 '尉',
 '廊',
 '任',
 '改',
 '法',
 '?',
 '艇',
 '李',
 'е',
 '癸',
 '池',
 '淸',
 '咬',
 '問',
 '癎',
 '特',
 '監',
 '孝',
 '項',
 '↑',
 '旗',
 'シ',
 '竪',
 '禁',
 '齒',
 '名',
 'Σ',
 '構',
 'フ',
 '續',
 '報',
 '隋',
 '入',
 '難',
 '港',
 '涿',
 'つ',
 '航',
 '番',
 '引',
 '樞',
 '＼',
 'イ',
 'П',
 '郡',
 '步',
 '光',
 '炎',
 '偉',
 '短',
 '硅'

In [28]:
left_context_sc= train_context_special_characters -train_answer_special_characters
# left_context_sc
for tmp in sorted(list(left_context_sc)):
    cnt += 1
    print(tmp, end=' ')
# len(set(''.join(processed_text[:,1])))

# $ & * ; = ? [ ] ^ _ ` { | } ± ² ´ µ ¹ ¾ É Ó × Þ ß à á æ ç è é ê ë ì í ð ó ö ù ú ü ý ā ă ć Đ đ ē ğ ī ł œ ū ŭ ơ ư ǫ ɔ ə ɛ ʲ ˈ ː ́ ΄ Α Δ Ε Κ Λ Μ Π Σ Φ ά έ ή ί α β γ δ ε η θ ι κ λ μ ν ξ ο π ρ ς τ υ φ χ ω ό ύ ώ І В Л М Н П Р С Ю а в д е з и й к л м н о п р с т х ч ш ы я і ִ ַ ׁ ׇ ח י מ ש إ ا ب ة ت ح ر ز س ش ص ع غ ق ك ل م ه و ي ی क त र ल स ़ ा ि ी ु ত র স া ে ச ஜ ம ய ர ழ ா ோ ் ა გ დ ე ვ თ ი ნ ო ს ც ხ ᐨ ᚢ ᚴ ᛁ ᛚ ạ ầ ờ ữ Ἰ ὁ ὔ ‎ – — ― • ․ … ‧ ℃ Ⅰ Ⅱ ← ↑ → ↓ ⇌ ∼ ① ② ③ ④ ⑤ ⑥ ⑦ ⑧ ⑨ ⑪ □ ▲ △ ○ ☆ ☞ ♣ 　 、 。 【 】 〔 〕 あ か が き こ ご さ だ ち つ づ て は び み め り る を ん ァ ア イ ウ ェ オ カ キ ク ゲ コ サ シ ジ ス タ ダ チ ッ ツ ト ド ニ ネ ノ ハ バ パ ヒ ビ ピ フ マ ム メ モ ャ ヨ ラ ル レ ロ ヲ ン ヶ ー ㅏ ㅕ ㅚ ㅟ ㆍ ㈜ ㎜ ㎝ ㎞ ㎠ ㎡ 丁 七 万 丈 三 上 下 不 丑 丕 世 丘 丙 丞 両 中 丸 丹 主 丽 乃 乘 乙 九 乱 乾 事 五 井 亡 交 亥 享 亭 人 仁 仇 今 他 仗 付 仙 代 令 仲 件 任 企 伊 伍 休 会 伯 伴 伸 伺 佃 位 低 住 体 何 佛 作 佩 使 來 例 侍 侗 依 侯 俊 俗 俘 保 俣 修 倉 倍 倒 候 借 倩 倫 偃 假 偉 停 健 側 傅 傍 備 傳 像 僑 僚 僧 價 儀 儒 償 兀 允 元 兆 先 光 克 兌 免 兒 党 兢 入 內 全 兩 兪 八 六 共 兴 兵 其 具 典 兼 内 円 再 冠 凱 凶 凸 凹 出 函 刑 列 初 判 別 利 制 刹 刺 則 剛 剝 副 創 劇 劍 劑 力 功 加 务 劣 助 勅 勇 勉 勒 動 勝 勞 勢 

In [29]:
ord(","), ord("，")

(44, 65292)

In [47]:
train_df["context"]

0       미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...
1       '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...
2       강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...
3       불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...
4       동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...
                              ...                        
3947    이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...
3948    애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...
3949    1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...
3950    원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...
3951    2008년 2월 28일 실시된 2008년 함부르크 주의회 선거에서 기민련은 과반수 ...
Name: context, Length: 3952, dtype: object

### 같은 의미를 가지지만 다른 기호를 쓴 것 들을 통일해주자

In [92]:
trian_special_char = train_context_special_characters | train_answer_special_characters
trian_special_char_str = " ".join(sorted(list(trian_special_char)))
print(len(trian_special_char_str))
print(trian_special_char_str)

4167
! " # $ % & ' ( ) * + , - . / : ; < = > ? [ ] ^ _ ` { | } ~ ­ ° ± ² ´ µ · ¹ ¾ É Ó × Þ ß à á ä æ ç è é ê ë ì í ð ó ö ù ú ü ý ā ă ć Đ đ ē ğ ī ł œ ū ŭ ơ ư ǫ ɔ ə ɛ ʲ ˈ ː ́ ΄ Α Δ Ε Κ Λ Μ Π Σ Φ ά έ ή ί α β γ δ ε η θ ι κ λ μ ν ξ ο π ρ ς τ υ φ χ ω ό ύ ώ І В Л М Н П Р С Ю а в д е з и й к л м н о п р с т х ч ш ы я і ִ ַ ׁ ׇ ח י מ ש إ ا ب ة ت ح ر ز س ش ص ع غ ق ك ل م ه و ي ی क त र ल स ़ ा ि ी ु ত র স া ে ச ஜ ம ய ர ழ ா ோ ் ა გ დ ე ვ თ ი ნ ო ს ც ხ ᐨ ᚢ ᚴ ᛁ ᛚ ạ ầ ờ ữ Ἰ ὁ ὔ ‎ – — ― ‘ ’ “ ” • ․ … ‧ ℃ Ⅰ Ⅱ ← ↑ → ↓ ⇌ ∧ ∼ ≪ ≫ ① ② ③ ④ ⑤ ⑥ ⑦ ⑧ ⑨ ⑪ □ ▲ △ ○ ☆ ☞ ♣ 　 、 。 〈 〉 《 》 「 」 『 』 【 】 〔 〕 あ い う か が き こ ご さ し だ ち つ づ て な に の は び み め よ ら り る を ん ァ ア イ ウ ェ オ カ キ ク ゲ コ サ シ ジ ス タ ダ チ ッ ツ ト ド ニ ネ ノ ハ バ パ ヒ ビ ピ フ マ ム メ モ ャ ヨ ラ ル レ ロ ヲ ン ヶ ・ ー ㅏ ㅕ ㅚ ㅟ ㆍ ㈜ ㎜ ㎝ ㎞ ㎠ ㎡ 一 丁 七 万 丈 三 上 下 不 丑 丕 世 丘 丙 丞 両 中 丸 丹 主 丽 乃 之 乘 乙 九 乱 乾 事 二 五 井 亡 交 亥 享 京 亭 人 仁 仇 今 他 仗 付 仙 代 令 仲 件 任 企 伊 伍 休 会 伯 伴 伸 伺 佃 位 低 住 体 何 佛 作 佩 使 來 例 侍 侗 依 侯 俊 俗 俘 保 信 俣 修 倉 倍 倒 候 借 倩 倫 偃 假 偉 停 健 側 傅 傍 備 傳 像 僑 僚 僧 價 儀 儒 償 優 儼 兀 允 元 兆 先 光 克 兌 免 兒 党 兢 入 內 全

### unknown token 확인

In [84]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [98]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [103]:
tokenizer.bos_token, tokenizer.sep_token, tokenizer.mask_token, tokenizer.additional_special_tokens
for obj in tokenizer.SPECIAL_TOKENS_ATTRIBUTES:
    print(getattr(tokenizer,obj))

<s>
</s>
<unk>
</s>
<pad>
<s>
<mask>
[]


In [97]:
tmp = []
for i in range(0,len(trian_special_char_str),512):
    tmp.extend(tokenizer.tokenize(trian_special_char_str[i:i+512]))
    # print(i)
print("<unk>" in tmp)
print(tmp)

False
['▁!', '▁"', '▁#', '▁$', '▁%', '▁&', "▁'", '▁(', '▁)', '▁*', '▁+', '▁', ',', '▁-', '▁', '.', '▁/', '▁:', '▁;', '▁<', '▁=', '▁>', '▁?', '▁[', '▁]', '▁^', '▁_', '▁`', '▁{', '▁', '|', '▁}', '▁~', '▁', '\xad', '▁°', '▁', '±', '▁2', '▁́', '▁μ', '▁·', '▁1', '▁', '3⁄4', '▁É', '▁Ó', '▁×', '▁Þ', '▁', 'ß', '▁à', '▁á', '▁ä', '▁æ', '▁ç', '▁è', '▁é', '▁ê', '▁', 'ë', '▁', 'ì', '▁í', '▁', 'ð', '▁ó', '▁ö', '▁', 'ù', '▁ú', '▁ü', '▁ý', '▁', 'ā', '▁', 'ă', '▁', 'ć', '▁Đ', '▁đ', '▁', 'ē', '▁', 'ğ', '▁', 'ī', '▁', 'ł', '▁', 'œ', '▁', 'ū', '▁', 'ŭ', '▁', 'ơ', '▁', 'ư', '▁', 'ǫ', '▁', 'ɔ', '▁ə', '▁', 'ɛ', '▁j', '▁', 'ˈ', '▁', 'ː', '▁́', '▁́', '▁Α', '▁Δ', '▁Ε', '▁Κ', '▁Λ', '▁Μ', '▁Π', '▁Σ', '▁Φ', '▁', 'ά', '▁έ', '▁ή', '▁', 'ί', '▁α', '▁β', '▁γ', '▁δ', '▁ε', '▁η', '▁θ', '▁', 'ι', '▁κ', '▁λ', '▁μ', '▁ν', '▁ξ', '▁ο', '▁π', '▁ρ', '▁', 'ς', '▁τ', '▁', 'υ', '▁φ', '▁χ', '▁', 'ω', '▁ό', '▁', 'ύ', '▁', 'ώ', '▁І', '▁В', '▁Л', '▁М', '▁Н', '▁П', '▁Р', '▁С', '▁Ю', '▁а', '▁в', '▁д', '▁е', '▁з', '▁и', '▁й', '▁к', '▁л'

In [89]:
len(tmp)

3928

In [78]:
tokenized_train_spe = " ".join(tokenizer.tokenize(trian_special_char_str))
tokenized_train_spe

'▁ !" # $ % & \' ( ) * + , - . / : ; < = > ? [ ] ^ _ ` { | } ~ \xad ° ± 2 ▁́ μ · 1 3⁄4 É Ó × Þ ß à á ä æ ç è é ê ë ì íð ó ö ù ú ü ý ā ă ć Đ đ ē ğ ī ł œ ū ŭ ơ ư ǫ ɔ ə ɛ j ˈ ː ́ ▁́ Α Δ ΕΚ Λ ΜΠ Σ Φ ά έ ή ία β γ δε ηθικ λμ ν ξο πρ ς τ υφ χω ό ύ ώ ІВ Л М НП РС Ю ав де з ий кл м но пр ст х ч шыя і ִ ַ ׁ ׇ חי מש إ ابة تحر ز س ش ص ع غ ق كل مه وي ی क तर लस ़ ा ि ी ु তর সা ে ச ஜ மய ர ழ ா ோ ் აგ დევ თი ნოს ც ხ ᐨᚢᚴᛁᛚ ạ ầ ờ ữ Ἰ ὁ ὔ ▁– — ― ‘ ’ “ ” • . ... ‧ ° C III ← ↑ → ↓ ⇌ ∧ ∼ ≪ ≫ 1234 56 789 11 □ ▲ △ ○ ☆ ☞ ♣ ▁ 、 。 〈 〉 《 》 「 」 『 』 【 】 〔 〕 あい う か が き こ ご さ し だ ち つ づ て な に のは び み め よ ら り る を ん ァ アイ ウェ オ カ キ ク ゲ コ サ シ ジ スタ ダ チ ッツ ト ド ニ ネ ノ ハ バ パ ヒ ビ ピ フ マ ム メモ ャ ヨ ラ ル レ ロ ヲ ン ヶ ・ ー ᅡ ᅧᅬᅱ ᆞ ( 주 ) mm cm km cm 2 m 2 一 丁 七 万 丈 三 上下 不 丑 丕 世 丘 丙 丞 両 中 丸 丹 主 丽 乃 之 乘 乙 九 乱 乾 事 二 五 井 亡 交 亥 享 京 亭 人 仁 仇 今 他 仗 付 仙 代 令 仲 件 任 企 伊 伍 休 会 伯 伴 伸 伺 佃 位 低 住 体 何 佛 作 佩 使 來 例 侍 侗 依 侯 俊 俗 俘 保 信 俣 修 倉 倍 倒 候 借 倩 倫 偃 假 偉 停 健 側 傅 傍 備 傳 像 僑 僚 僧 價 儀 儒 償 優 儼 兀 允 元 兆 先 光 克 兌 免 兒 党 兢 入 內 全 兩 兪 八 公 六 共 兴 兵 其 具 典 兼 内 円 再 冠 冬 凱 凶 凸 凹 出

In [71]:
unk_chars = []
trian_special_char_str = sorted(list(trian_special_char))
tokenized_train_spe = tokenized_train_spe.split(" ")
for origin, processed in zip( trian_special_char_str, tokenized_train_spe):
    if processed == "[UNK]":
        unk_chars.append(origin)
unk_chars

['\xad',
 '°',
 '´',
 '·',
 '¹',
 '¾',
 'É',
 '×',
 'Þ',
 'ß',
 'à',
 'á',
 'ä',
 'æ',
 'ç',
 'é',
 'ê',
 'ë',
 'ì',
 'í',
 'ð',
 'ó',
 'ö',
 'ù',
 'ú',
 'ü',
 'ý',
 'ā',
 'ă',
 'ć',
 'Đ',
 'đ',
 'ē',
 'ğ',
 'ī',
 'ł',
 'œ',
 'ū',
 'ŭ',
 'ơ',
 'ư',
 'ǫ',
 'ɔ',
 'ə',
 'ɛ',
 'ʲ',
 'ˈ',
 'ː',
 '́',
 '΄',
 'Α',
 'Δ',
 'Ε',
 'Κ',
 'Λ',
 'Μ',
 'Π',
 'Σ',
 'Φ',
 'ά',
 'έ',
 'ή',
 'ί',
 'α',
 'β',
 'γ',
 'δ',
 'ε',
 'η',
 'θ',
 'ι',
 'κ',
 'λ',
 'μ',
 'ν',
 'ξ',
 'ο',
 'π',
 'ρ',
 'ς',
 'τ',
 'υ',
 'φ',
 'χ',
 'ω',
 'ό',
 'ύ',
 'ώ',
 'І',
 'В',
 'Л',
 'М',
 'Н',
 'П',
 'Р',
 'С',
 'а',
 'в',
 'е',
 'и',
 'й',
 'к',
 'л',
 'о',
 'т',
 'х',
 'ч',
 'ш',
 'ы',
 'я',
 'і',
 'ִ',
 'ַ',
 'ׁ',
 'ׇ',
 'ח',
 'י',
 'מ',
 'ש',
 'إ',
 'ا',
 'ب',
 'ة',
 'ت',
 'ح',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ع',
 'غ',
 'ق',
 'ك',
 'ل',
 'م',
 'ه',
 'و',
 'ي',
 'ی',
 'क',
 'त',
 'र',
 'ल',
 'स',
 '़',
 'ा',
 'ि',
 'ी',
 'ु',
 'ত',
 'র',
 'স',
 'া',
 'ে',
 'ச',
 'ஜ',
 'ம',
 'ய',
 'ர',
 'ழ',
 'ா',
 'ோ',
 '்',
 'ა',
 'გ',
 

In [72]:
" ".join(unk_chars)

'\xad ° ´ · ¹ ¾ É × Þ ß à á ä æ ç é ê ë ì í ð ó ö ù ú ü ý ā ă ć Đ đ ē ğ ī ł œ ū ŭ ơ ư ǫ ɔ ə ɛ ʲ ˈ ː ́ ΄ Α Δ Ε Κ Λ Μ Π Σ Φ ά έ ή ί α β γ δ ε η θ ι κ λ μ ν ξ ο π ρ ς τ υ φ χ ω ό ύ ώ І В Л М Н П Р С а в е и й к л о т х ч ш ы я і ִ ַ ׁ ׇ ח י מ ש إ ا ب ة ت ح ر ز س ش ص ع غ ق ك ل م ه و ي ی क त र ल स ़ ा ि ी ु ত র স া ে ச ஜ ம ய ர ழ ா ோ ் ა გ დ ე ვ თ ი ნ ო ს ც ხ ᐨ ᚢ ᚴ ᛁ ᛚ ạ ầ ờ ữ Ἰ ὁ ὔ Ⅰ ↑ → ↓ ⑥ ⑦ ⑧ ☆ ☞ ♣ 】 〔 〕 あ い う か が き こ ご さ し だ ち つ づ な に の は び み め よ ら り る を ん ァ ア イ ウ ェ オ カ キ ク ゲ コ サ シ ジ ス タ ダ チ ッ ツ ト ド ニ ネ ノ ハ バ パ ヒ ビ ピ フ マ ム メ モ ャ ヨ ラ ル レ ロ ン ・ ー ㅏ ㎜ ㎠ 一 丁 上 下 丑 丕 世 丘 丞 両 丸 丹 丽 乃 乘 乙 事 二 五 井 亡 亥 亭 人 仁 仇 今 他 付 仙 代 仲 件 任 企 伊 伍 休 会 伯 伴 伸 伺 佃 位 低 何 使 來 例 侍 侗 依 侯 俘 信 俣 修 倉 倍 倒 候 借 倩 倫 偃 假 偉 停 健 側 傍 備 傳 像 僑 僚 僧 價 儀 儒 償 優 兀 兆 先 光 克 兌 免 入 內 兪 公 兴 其 具 典 兼 内 円 再 冠 冬 凱 凸 凹 函 刀 分 別 利 刺 則 前 剛 剝 副 割 創 劇 劍 劑 力 功 加 务 劣 助 勅 勒 動 務 勝 勞 勢 包 化 北 匠 匡 区 區 十 千 午 卓 協 南 博 卜 卞 卯 印 危 卵 厚 原 厲 参 叉 友 叔 叙 口 句 可 吉 同 名 吏 向 君 否 吳 呉 告 呼 命 和 咎 品 哉 唐 問 啓 善 喜 喪 喬 單 嗣 囊 囚 四 回 国 圈 國 園 團 在 圭 地 坂 均 坊 坐 坮 型 域 基 堤 報 場 堵 塑 塔 塗 塘 塚 塡 

## Valid 살펴보기

In [30]:
ids = []
answers = []
contexts = []
questions = []
titles = []
document_ids = []

for ex in dataset['validation']:
    ids.append(ex['id'])
    answers.append(ex['answers'])
    contexts.append(ex['context'])
    questions.append(ex['question'])
    titles.append(ex['title'])
    document_ids.append(ex['document_id'])


valid_df = pd.DataFrame({"id": ids,"question" : questions, "answer": answers, "context" : contexts, "title" : titles, "doc_id" : document_ids})


In [31]:
valid_df

,id,question,answer,context,title,doc_id
0,mrc-0-003264,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,"{'answer_start': [284], 'text': ['한보철강']}","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",전효숙,9027
1,mrc-0-004762,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,"{'answer_start': [146], 'text': ['1871년']}",요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스토우빌선,51765
2,mrc-1-001810,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,"{'answer_start': [517], 'text': ['나뭇잎']}","촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촐라,15694
3,mrc-1-000219,로타이르가 백조를 구하기 위해 사용한 것은?,"{'answer_start': [1109], 'text': ['금대야']}",프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,백조 기사,59536
4,mrc-1-000285,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,"{'answer_start': [386], 'text': ['수평적 관계']}",탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,탈관료제화,32991
...,...,...,...,...,...,...
235,mrc-0-000484,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,"{'answer_start': [1084], 'text': ['제 양왕']}","연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...",전단 (제나라),53265
236,mrc-0-002095,공놀이 경기장 중 일부는 어디에 위치하고 있나?,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장...",현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,티칼,12190
237,mrc-0-003083,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,"{'answer_start': [247], 'text': ['미나미 지로']}",1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,윤치호,5326
238,mrc-0-002978,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,"{'answer_start': [563], 'text': ['200,000명']}",빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,신잉카국,48575


In [32]:
valid_df.describe()

,doc_id
count,240.000000
mean,29027.704167
std,17915.733034
min,4866.000000
25%,11726.250000
50%,27485.000000
75%,44860.250000
max,60305.000000


In [33]:
ans_text = []
for ans in valid_df["answer"]:
    ans_text.append(*ans['text'])
ans_text

['한보철강',
 '1871년',
 '나뭇잎',
 '금대야',
 '수평적 관계',
 '옥음방송',
 '코칭 스티치',
 '복잡한 감염병',
 '스페인',
 '20세기 초',
 '"5월의 왕"',
 "'일급 비밀 프로젝트 2501'",
 '테헤란',
 '역사교육과정개발추진위원회',
 '1967년 11월 15일',
 '1965년',
 '아리크 부케',
 '〈중앙일보〉',
 '미타케성',
 '전체 4순위',
 '뇌물',
 '보통 유형 준융합성 천연두',
 '데스탱 장군',
 '박트리아',
 "'진전(陳田)'이라 새겨진 기와조각",
 '《국가》',
 '2011년 3월 19일',
 '유럽 포르투갈어',
 '메이저 릿지',
 '요희',
 '포드 극장',
 '만주국 관리',
 '『협동조합에 관하여』',
 '도고쿠',
 '몽키 D. 가프',
 '조류',
 '광주교도소',
 '작가 베게티우스',
 '가오슝 시',
 '쇠망치',
 '마리즈 교수',
 '런던',
 '이틀',
 '의상대사',
 '홋카이도',
 '‘우유의 바다’',
 '장어',
 '소련',
 '전치',
 '흑색육(黑色肉)',
 '욱',
 '독일인민당',
 '하기노 역',
 '회칠',
 '황강다리',
 '직업 교육',
 '공산당',
 '매년 음력 정월',
 '조계종',
 '베이징',
 '독일군',
 '1932년',
 '제서지전(齊西之戰)이후',
 '가루아',
 '출생 천궁도',
 '나말여초',
 '1939년',
 '창 절제술',
 '도버 밀',
 '여정현(呂正鉉)',
 '광배',
 '울산',
 '땅(은색)과 하늘(금색)을 맺고 푸는 권한',
 '저수지',
 '퀘이커 교',
 '피에르 오주로',
 '지분',
 '독일',
 '제 3자',
 '페이팔',
 '가리타 히사노리',
 '점수',
 '코스모케라톱스',
 "'사채회사'",
 '스위스',
 '"공유지의 비극"',
 '그 자체가 계급 모순을 가지는 체제',
 '해리 트루먼 대통령',
 '무위태수',
 '크라운라이터',
 '1990년 9월',
 '"인간과 요괴의 완전한 평

In [34]:
processed_text = []
for idx, text in enumerate(ans_text):
    tmp = re.sub(r"[가-힣]+|[0-9]+|[ ]+|[a-z]+|[A-Z]+|[ㄱ-ㅎ]+","",text)
    if tmp != '':
        processed_text.append((idx,tmp))
processed_text

[(10, '""'),
 (11, "''"),
 (17, '〈〉'),
 (24, "'(陳田)'"),
 (25, '《》'),
 (32, '『』'),
 (34, '.'),
 (45, '‘’'),
 (49, '(黑色肉)'),
 (62, '(齊西之戰)'),
 (69, '(呂正鉉)'),
 (72, '()()'),
 (83, "''"),
 (85, '""'),
 (91, '""'),
 (99, '.'),
 (109, ','),
 (119, '()'),
 (146, '“”'),
 (151, '()'),
 (152, '()'),
 (160, '(:)'),
 (184, '《》'),
 (186, '·()'),
 (198, '(ć)'),
 (201, "''()"),
 (204, '＜＞'),
 (215, '""()'),
 (229, '(楊黨)'),
 (234, '(物的成果)'),
 (236, "''"),
 (238, ',')]

In [35]:
processed_text = np.array(processed_text)
print(len(set(processed_text[:,1])))
for text in set(processed_text[:,1]):
    print(text,end=", ")
# print(set(processed_text[:,1]))


23
“”, (ć), 〈〉, (黑色肉), ., (:), 『』, ‘’, ·(), ,, ''(), ＜＞, 《》, (呂正鉉), (物的成果), (楊黨), '', "", (), '(陳田)', ()(), ""(), (齊西之戰), 

In [36]:
valid_df.iloc[72]

id                                               mrc-0-002692
question                   교황의 문장에서 교차한 금빛 열쇠와 은빛 열쇠가 뜻하는 바는?
answer      {'answer_start': [504], 'text': ['땅(은색)과 하늘(금색...
context     교황 문장은 기독교에서 교황을 상징하는 고유한 문장을 말한다. 최초로 문장을 사용한...
title                                                   교황 문장
doc_id                                                  19457
Name: 72, dtype: object

In [37]:
cnt = 0
for tmp in sorted(list(set(''.join(processed_text[:,1])))):
    cnt += 1
    print(tmp, end=' ')
print()
print(cnt)

" ' ( ) , . : · ć ‘ ’ “ ” 〈 〉 《 》 『 』 之 呂 成 戰 果 楊 正 物 田 的 肉 色 西 鉉 陳 黑 黨 齊 ＜ ＞ 
39


In [38]:
len('\" \' ( ) , . : · ć ‘ ’ “ ” 〈 〉 《 》 『 』 之 呂 成 戰 果 楊 正 物 田 的 肉 色 西 鉉 陳 黑 黨 齊 ＜ ＞'.split(' '))


39

### answer 특수문자 살펴보기

In [39]:
valid_answer_special_characters = set(''.join(processed_text[:,1]))
len(valid_answer_special_characters)

39

### context에 존재하는 특수문자 살펴보기

In [40]:
valid_contexts = valid_df["context"]

In [41]:
processed_text = []
for idx, text in enumerate(valid_contexts):
    tmp = re.sub(r"\\n+|\'|[가-힣]+|[0-9]+|[ ]+|[a-z]+|[A-Z]+|[ㄱ-ㅎ]+","",text)
    if tmp != '':
        processed_text.append((idx,tmp))
processed_text

[(0, ',,,,..(""""...,.'),
 (1, '&.....,&..,.....'),
 (2, ',.,.(),(),(),(),,.,··,.'),
 (3, '..《》().),),),).《》().(-.),.,.,..,.,..,.......'),
 (4, '.,.·.,.,..·,.,..,,.'),
 (5, '()""(皇国義勇軍)..,,(),,,.,().."",.,,.(:),.,,.,.,²...·,,,.'),
 (6, ',,.,,,,···,,,(量感),,,,,,,,·.'),
 (7, ',/.,.(,)(,).,...'),
 (8, '(-).............~.........'),
 (9, '.......()-...'),
 (10, '...,.,.,"".,....,.,..,.,.'),
 (11, '.,.,,..,..,..,,,,..,.,(多脚),,,.'),
 (12, '...,.·,,,..,,.()..""."".......'),
 (13, ',"ㆍ"""""."""".(案).""."".."""""".()“,”..()“‘’‘’,”.'),
 (14, '.,..,..,""().,,,.,,,,.().=|"".""""."".=|"""","".'),
 (15, ',.,,,%%...........'),
 (16, '(,孛兒只斤阿里不哥|,~).,,.,.(),(開平府:)..,(帝都),...,.,,..'),
 (17, '·,,,,,,[『』『』.∼..『』『』.『』『』.]〈〉『』“”'),
 (18, ',.,.,,..,「」.,,,(,).,...,(,).(),..,.()..(,).,..,「」.,,,,.,.,.。,.『』.,。'),
 (19, '.,...,.,...'),
 (20, ',,.,""."",.,,.,",,".,."",",,".'),
 (21, '%,.~%,~%.%.~%..%%.%,~.,.,,.,.,,··,.'),
 (22, ',....,.,.‘’,.,.,.'),
 (23, ',..,,..(),.,..,.,,,.,.,.,,.,.,.,..,..,..'),
 (24, '.,(陳田).

In [42]:
processed_text = np.array(processed_text)
cnt=0
for tmp in sorted(list(set(''.join(processed_text[:,1])))):
    cnt += 1
    print(tmp, end=' ')
len(set(''.join(processed_text[:,1])))

! " # $ % & ( ) * + , - . / : ; < = > ? [ ] | ~ ° ² µ · ¾ è é ö ü ć α β Л а в е и л о п р с ц ч ட ந ப ல வ ா ு ் – ‘ ’ “ ” … ℓ ∼ ≪ ≫ 、 。 〈 〉 《 》 「 」 『 』 あ が し す で に の は み ゆ ら り ア カ ス プ リ ル ・ ㆍ ㎝ ㎞ 一 三 上 下 不 丘 中 丹 主 丼 乃 久 之 乱 事 五 亞 交 亭 人 他 令 以 任 会 佐 佛 侍 侯 信 修 個 候 傳 像 僚 優 先 光 兒 全 八 公 共 冬 冷 初 前 劉 勇 化 北 千 南 印 原 叔 取 只 吉 同 君 呂 周 味 哥 器 国 國 圖 圭 地 城 基 堞 報 場 塔 塡 壇 多 夢 大 天 奏 女 妙 姬 子 孛 季 孫 學 安 宗 宮 寺 尉 尊 小 少 尹 尼 川 州 左 巨 市 希 帝 带 師 帶 常 平 府 度 座 建 式 張 形 後 志 念 性 悟 情 感 慶 成 我 戟 戰 戶 手 拳 捨 政 敏 文 斐 斤 昌 明 春 會 朝 木 未 本 札 李 杏 杜 東 林 果 校 株 桀 案 梅 梯 梵 森 楊 業 樂 標 樹 機 櫃 欽 正 武 殷 毌 氏 氣 水 永 池 洋 流 济 淖 淵 混 湯 溢 滅 漸 濟 火 炎 炭 然 牛 物 狂 狭 玄 玉 王 球 瑞 瑾 生 田 百 的 皇 相 真 石 社 神 禅 福 移 紀 紂 素 紳 終 經 繼 羌 義 聖 聯 聲 肅 肉 背 脚 臣 臺 興 舞 舟 航 色 花 若 英 莒 華 萬 藏 藤 虎 虹 衆 行 衝 衣 複 西 覚 親 觀 計 說 諡 護 讚 谷 財 赤 路 踊 身 軍 辛 通 速 進 運 道 遠 遼 郞 部 都 里 重 量 金 鈴 鉉 錯 鑊 長 門 開 閔 阿 陳 陸 陽 隠 雄 雉 雨 震 非 面 須 高 髮 魏 魔 鮮 鰻 黃 黎 黑 黒 點 黨 鼓 齊 齋 － ＜ ＞ Ｍ Ｓ 

428

## Wiki data 살펴보기

In [6]:
wiki_raw_df = pd.read_json(WIKI_JSON_PATH)

In [7]:
wiki_raw_df = wiki_raw_df.T
wiki_raw_df

,text,title,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",나라 목록,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,나라 목록,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,백남준,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",아오조라 문고,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,아오조라 문고,4
...,...,...,...
60608,"오키나와 현립 박물관·미술관( , Okiawa Peecual Museum ad A ...",오키나와 현립 박물관·미술관,60608
60609,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",오키나와 현립 박물관·미술관,60609
60610,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,오키나와 현립 박물관·미술관,60610
60611,단결권 및 단체교섭권 협약(Righ o Ogaise ad Collecie Bagai...,단결권 및 단체교섭권 협약,60611


In [8]:
wiki_raw_df.iloc[1004]

text           바누아투는 고고학적 증거로 약 기원전 45,000년 부터 1,500년 까지 남태평양...
title                                                       바누아투
document_id                                                 1004
Name: 1004, dtype: object

In [45]:
wiki_raw_df[wiki_raw_df["document_id"]==15575]["text"] == train_df.iloc[544]["context"]

15575    True
Name: text, dtype: bool

In [46]:
wiki_raw_df["document_id"].unique()

array([0, 1, 2, ..., 60610, 60611, 60612], dtype=object)